In [ ]:
## Done

In [ ]:
from google.colab import drive; drive.mount("/content/drive")

In [ ]:
import pandas as pd
import numpy as np
import nltk 
import re
nltk.download("stopwords");nltk.download("punkt")
import sklearn 

In [ ]:
data1 = pd.read_excel("/content/drive/My Drive/DimasASu/Data Latih BDC.xlsx")
data2 = pd.read_ csv("/content/drive/My Drive/DimasASu/data_clean.csv")

dataTest = pd.read_csv("/content/drive/My Drive/DimasASu/datatest_labelled.csv")

data1.head() , data2.head()

In [ ]:
data1 = data1[["judul","narasi","label"]]
data2 = data2[["judul","narasi","label"]]

df_train = pd.concat([data1,data2])
df_train = df_train.reset_index().drop("index",axis = 1)
df_train.tail()

In [ ]:
def clean(text_messages):
    text_messages = str(text_messages)
    processed = re.sub(r'^.+@[^\.].*\.[a-z]{2,}$',
                                    'alamat email',text_messages)

    # Replace URLs with 'webaddress'
    processed = re.sub(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                      'alamat web',processed)

    # Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
    processed = re.sub(r'£|\$', 'moneysymb',processed)
        
    # Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
    processed = re.sub(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                      'nomor hp',processed)
        
    # Replace numbers with 'numbr'
    processed = re.sub(r'\d+(\.\d+)?', 'numbr',processed)




    # Remove punctuation
    processed = re.sub(r'[^\w\d\s]', ' ',processed)

    # Replace whitespace between terms with a single space
    processed = re.sub(r'\s+', ' ',processed)

    # Remove leading and trailing whitespace
    processed = re.sub(r'^\s+|\s+?$', '',processed)

    processed = processed.lower()

    return processed

In [ ]:
df_train["narasi"] = df_train["narasi"].apply(clean)
df_train["judul"] = df_train["judul"].apply(clean)

In [ ]:
from nltk.corpus import stopwords

stopwordsObj = set(stopwords.words("indonesian"))

df_train["narasi"] = df_train["narasi"].apply(lambda sentence: " ".join(word for word in sentence.split(" ") if word not in stopwordsObj))
df_train["judul"] = df_train["judul"].apply(lambda sentence: " ".join(word for word in sentence.split(" ") if word not in stopwordsObj))

In [ ]:
porterStemmer = nltk.PorterStemmer()
df_train["narasi"] = df_train["narasi"].apply(lambda sentence: " ".join(porterStemmer.stem(word) for word in sentence.split(" ")) )

df_train["narasi"].head()

In [ ]:
from nltk.tokenize import word_tokenize

corpus = []

for message in df_train["narasi"]:
  words = word_tokenize(message)
  for word in words:
    corpus.append(word)

for message in df_train["judul"]:
  words = word_tokenize(message)
  for word in words:
    corpus.append(word)

In [ ]:
corpus = nltk.FreqDist(corpus)
corpus

In [ ]:
word_features = list(corpus.keys())[:5000]

In [ ]:
def find_features(message):
  message = str(message)
  words = word_tokenize(message)
  features = {}
  for word in word_features:
    features[word] = word in words
  
  return features

In [ ]:
messages = list(zip(df_train["narasi"],df_train["label"]))
np.random.seed(1)
print(messages)
featuresets = [(find_features(text),label) for (text,label) in messages] 

In [ ]:
featuresets[:5]

In [ ]:
from sklearn.model_selection import train_test_split
training,testing = train_test_split(featuresets,test_size = 0.05,random_state = 1)

print(len(training), len(testing))

In [ ]:
from nltk import SklearnClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

In [ ]:
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))
nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))